# Learning Analytics Package: Build Fact Tables

Builds the fact table used for the Learning Analytics package **v1.0** dashboard, in the context of using the Higher Ed. test data from Microsoft Education Insights data, and Microsoft Graph meeting attendance data.

The following tables are created in each of the steps outlined below:

1. fact_Enrollment
2. fact_MeetingAttendance
3. fact_Assignment
4. fact_Activity

This package-notebook also uses two methods (defined and outlined below; same as the LA_build_dimension_tables notebook):
 - **_publish_to_stage2**: uses the OEA_py function *upsert* to land the current dataframe in stage2/Enriched
 - **publish**: uses the method above to land the package fact tables in stage2, then writes the same table to stage3/Published. Delta checkpoints are landed in the respective stage2/Enriched directory.

In [1]:
workspace = 'dev'

StatementMeta(spark3p3sm, 113, 1, Finished, Available)

In [2]:
%run OEA_py

StatementMeta(, 113, -1, Finished, Available)

2023-02-27 02:40:34,185 - OEA - INFO - Now using workspace: dev
2023-02-27 02:40:34,186 - OEA - INFO - OEA initialized.


In [3]:
# set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace(workspace)

StatementMeta(spark3p3sm, 113, 3, Finished, Available)

2023-02-27 02:40:34,494 - OEA - INFO - Now using workspace: dev


## 0.) Define the Publish Function

In [4]:
def _publish_to_stage2(df, destination, pk):
    oea.upsert(df, destination, pk)

def publish(df, stage2_destination, stage3_destination, primary_key='id'):
    _publish_to_stage2(df, stage2_destination, primary_key)

    spark.sql("set spark.sql.streaming.schemaInference=true")
    streaming_df = spark.readStream.format('delta').load(oea.to_url(stage2_destination))
    # for more info on append vs complete vs update modes for structured streaming: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#basic-concepts
    query = streaming_df.writeStream.format('delta').outputMode('append').trigger(once=True).option('checkpointLocation', oea.to_url(stage2_destination) + '/_checkpoints')
    query = query.start(oea.to_url(stage3_destination))
    query.awaitTermination()   # block until query is terminated, with stop() or with error; A StreamingQueryException will be thrown if an exception occurs.
    number_of_new_inbound_rows = query.lastProgress["numInputRows"]
    logger.info(f'Number of new inbound rows processed: {number_of_new_inbound_rows}')
    logger.debug(query.lastProgress)
    return number_of_new_inbound_rows

StatementMeta(spark3p3sm, 113, 4, Finished, Available)

## 1.) Build fact_Enrollment Table

Data aggregations and curation on Insights roster data: Enrollment (as well as others).

This table has one row per student enrollment in a section from the Insights roster data, with details around:
 - enrollment ID (unique ID per student enrolled in a section),
 - school ID (Id from Organization table),
 - course ID (course that the section belongs to),
 - section ID,
 - instructor ID (that section's instructor)
 - student ID (previously, student internal ID or PersonId from Person table), and
 - student section entry and exit date (from Insights Enrollment table).

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/fact_Enrollment```.

**NOTE(s):**
 - If using test data, this process may result in null entry and exit date columns. This should not be needed to support the Power BI dashboard.
 - If using production data, these columns should not be nulled - but if they are, you may need to edit the Insights module ingestion/refinement process.

In [5]:
dfInsights_aadgroup = oea.load('stage2/Refined/M365/v1.14/general/AadGroup')
dfInsights_activity = oea.load('stage2/Refined/M365/v1.14/general/activity')
dfInsights_enrollment = oea.load('stage2/Refined/M365/v1.14/general/Enrollment')
dfInsights_section = oea.load('stage2/Refined/M365/v1.14/general/Section')
# lookup(s)
dfInsights_aadgroup_np = oea.load('stage2/Refined/M365/v1.14/sensitive/AadGroup_lookup')

StatementMeta(spark3p3sm, 113, 5, Finished, Available)

In [6]:
# NOTE: Fix entry and exit date column-processing, and clean up curation process below.
# extract the classes initally provided from the Insights Activity test data, and extract the Teacher IDs 
dfEnroll = dfInsights_activity.filter(dfInsights_activity['ActorRole'] != 'Student')
dfEnroll = dfEnroll.filter(dfEnroll['ActorRole'] != 'ActorRole')
dfEnroll = dfEnroll.select('ClassId', 'ActorId_pseudonym', 'ActorRole')
dfEnroll = dfEnroll.groupBy('ClassId', 'ActorId_pseudonym', 'ActorRole').count()
dfEnroll = dfEnroll.withColumnRenamed('ClassId', 'AADGroup_ClassId').withColumnRenamed('ActorId_pseudonym', 'InstructorId_pseudonym').withColumnRenamed('ActorRole', 'PersonRole')
dfEnroll = dfEnroll.drop('count')
# join the Insights AADGroup_pseudo and _lookup table to the dfEnroll table, to provide mapping of the hashed and non-hashed AADGroup Class IDs
dfInsights_aadgroup_np_ = dfInsights_aadgroup_np.select('ObjectId', 'DisplayName', 'ObjectId_pseudonym').withColumnRenamed('ObjectId', 'Id').withColumnRenamed('ObjectId_pseudonym', 'AADGroup_ClassId_pseudonym')
dfEnroll = dfEnroll.join(dfInsights_aadgroup_np_, dfEnroll.AADGroup_ClassId == dfInsights_aadgroup_np_.Id, how='inner')
dfEnroll = dfEnroll.drop('Id')
dfInsights_aadgroup_ = dfInsights_aadgroup.select('ObjectId_pseudonym', 'SectionId')
dfEnroll = dfEnroll.join(dfInsights_aadgroup_, dfEnroll.AADGroup_ClassId_pseudonym == dfInsights_aadgroup_.ObjectId_pseudonym, how='inner')
dfEnroll = dfEnroll.drop('ObjectId_pseudonym', 'PersonRole', 'DisplayName', 'AADGroup_ClassId', 'AADGroup_ClassId_pseudonym')
# use the Insights Section table to get each associated Course and School ID with a Section
df_metadata = dfInsights_section.select('Id', 'OrganizationId', 'CourseId')
dfEnroll = dfEnroll.join(df_metadata, dfEnroll.SectionId == df_metadata.Id, how='inner').drop('Id')
dfEnroll = dfEnroll.withColumnRenamed('OrganizationId', 'SchoolId')
# last, use the Insights Enrollment table to get students enrolled within each class
df_metadata = dfInsights_enrollment.select('Id', 'PersonId_pseudonym', 'RefSectionRoleId', 'SectionId', 'EntryDate', 'ExitDate')
df_metadata = df_metadata.filter(df_metadata['RefSectionRoleId'] == 'Student')
df_metadata = df_metadata.drop('RefSectionRoleId').withColumnRenamed('Id', 'EnrollmentId').withColumnRenamed('PersonId_pseudonym', 'StudentId_pseudonym').withColumnRenamed('SectionId', 'id')
dfEnroll = dfEnroll.join(df_metadata, dfEnroll.SectionId == df_metadata.id, how='inner').drop('id')
dfEnroll = dfEnroll.select('EnrollmentId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'EntryDate', 'ExitDate')
display(dfEnroll.limit(10))

StatementMeta(spark3p3sm, 113, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7a1fd458-89a0-4f5e-8575-3ebf7ec455cc)

### Write to Stage 3

In [54]:
publish(dfEnroll, 'stage2/Enriched/learning_analytics/v1.0/general/fact_Enrollment', 'stage3/Published/learning_analytics/v1.0/general/fact_Enrollment', primary_key='EnrollmentId')

StatementMeta(spark3p3sm, 59, 7, Finished, Available)

2023-01-11 18:55:11,111 - OEA - INFO - Number of new inbound rows processed: 3600


3600

## 2.) Build fact_MeetingAttendance Table

Data aggregations curation on Insights activity data and Graph meeting_attendance_report pertaining to meetings (as well as using a few other tables).

This table has one row per student per meeting recorded from in both Insights and from the Graph query, with details around:
 - meeting attendance ID (unique ID per student per meeting - scraped from Insights Activity SignalId),
 - school ID (ID from Organization table),
 - course ID, 
 - section ID, 
 - instructor ID, 
 - student ID (from Person table),
 - meeting ID (unique ID per meeting - same across Insights activity and Graph meeting query),
 - student join time, 
 - student leave time,
 - student attendance time in seconds.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/fact_MeetingAttendance```.

In [55]:
dfInsights_aadgroup = oea.load('stage2/Refined/M365/v1.14/general/AadGroup')
dfInsights_aaduserpersonmap = oea.load('stage2/Refined/M365/v1.14/general/AadUserPersonMapping')
dfInsights_activity = oea.load('stage2/Refined/M365/v1.14/general/activity')
dfInsights_enrollment = oea.load('stage2/Refined/M365/v1.14/general/Enrollment')
dfGraph_meetingAtten = oea.load('stage2/Refined/graph_api/v1.0/general/meeting_attendance_report')
# lookup(s)
dfInsights_aadgroup_np = oea.load('stage2/Refined/M365/v1.14/sensitive/AadGroup_lookup')

StatementMeta(spark3p3sm, 59, 8, Finished, Available)

In [56]:
# grab only the relevant columns from the Graph meeting attendance report data, then join with the Insights AADUserPersonMapping to join with Insights activity data
dfMeetingAtten = dfGraph_meetingAtten.filter(dfGraph_meetingAtten['role'] == 'Student').select('userId_pseudonym', 'meetingId', 'attendanceInterval_joinDateTime', 'attendanceInterval_leaveDateTime', 'totalAttendanceInSec')
dfMeetingAtten = dfMeetingAtten.withColumnRenamed('userId_pseudonym', 'StudentId_pseudonym').withColumnRenamed('meetingId', 'MeetingId').withColumnRenamed('attendanceInterval_joinDateTime', 'JoinTime') \
                                .withColumnRenamed('attendanceInterval_leaveDateTime', 'LeaveTime').withColumnRenamed('totalAttendanceInSec', 'AttendanceTime_sec')
df_metadata = dfInsights_aaduserpersonmap.select('ObjectId_pseudonym', 'PersonId_pseudonym')
dfMeetingAtten = dfMeetingAtten.join(df_metadata, dfMeetingAtten.StudentId_pseudonym == df_metadata.PersonId_pseudonym, how='inner').drop('PersonId_pseudonym')
# then join with Insights activity data to isolate meeting data
df_metadata = dfInsights_activity.filter(dfInsights_activity['AppName'] == 'TeamsMeeting').filter(dfInsights_activity['ActorRole'] == 'Student')
df_metadata = df_metadata.select('SignalId', 'ClassId', 'ActorId_pseudonym', 'MeetingSessionId')
dfMeetingAtten = dfMeetingAtten.join(df_metadata, (dfMeetingAtten.MeetingId == df_metadata.MeetingSessionId) & (dfMeetingAtten.ObjectId_pseudonym == df_metadata.ActorId_pseudonym), how='inner') \
                            .drop('ActorId_pseudonym', 'MeetingSessionId', 'ObjectId_pseudonym').withColumnRenamed('SignalId', 'MeetingAttendanceId')
# join the Insights AADGroup_pseudo and _lookup table to the dfMeetingAtten table, to provide mapping of the hashed and non-hashed AADGroup Class IDs
dfInsights_aadgroup_np_ = dfInsights_aadgroup_np.select('ObjectId', 'ObjectId_pseudonym').withColumnRenamed('ObjectId', 'Id').withColumnRenamed('ObjectId_pseudonym', 'AADGroup_ClassId_pseudonym')
dfMeetingAtten = dfMeetingAtten.join(dfInsights_aadgroup_np_, dfMeetingAtten.ClassId == dfInsights_aadgroup_np_.Id, how='inner').drop('Id')
dfInsights_aadgroup_ = dfInsights_aadgroup.select('ObjectId_pseudonym', 'SectionId')
dfMeetingAtten = dfMeetingAtten.join(dfInsights_aadgroup_, dfMeetingAtten.AADGroup_ClassId_pseudonym == dfInsights_aadgroup_.ObjectId_pseudonym, how='inner') \
                            .drop('ClassId', 'AADGroup_ClassId_pseudonym', 'ObjectId_pseudonym')
# finally, join dfMeetingAtten with dfEnroll table to grab the school, course and instructor IDs
df_metadata = dfEnroll.select('SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym')
df_metadata = df_metadata.withColumnRenamed('SectionId', 'classId').withColumnRenamed('StudentId_pseudonym', 'id')
dfMeetingAtten = dfMeetingAtten.join(df_metadata, (dfMeetingAtten.StudentId_pseudonym == df_metadata.id) & (dfMeetingAtten.SectionId == df_metadata.classId), how='inner').drop('classId', 'id')
dfMeetingAtten = dfMeetingAtten.select('MeetingAttendanceId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'MeetingId', 'JoinTime', 'LeaveTime', 'AttendanceTime_sec')
display(dfMeetingAtten.limit(10))

StatementMeta(spark3p3sm, 59, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, d4a37f2f-6f30-4d10-9e18-94bfd6802c11)

### Write to Stage 3

In [57]:
publish(dfMeetingAtten, 'stage2/Enriched/learning_analytics/v1.0/general/fact_MeetingAttendance', 'stage3/Published/learning_analytics/v1.0/general/fact_MeetingAttendance', primary_key='MeetingAttendanceId')

StatementMeta(spark3p3sm, 59, 10, Finished, Available)

2023-01-11 18:57:25,115 - OEA - INFO - Number of new inbound rows processed: 10170


10170

## 3.) Build fact_Assignment Table

Data aggregations curation on Insights activity data pertaining to assignments in the education system (as well as using a few other tables).

This table has one row per student per assignment from the Insights activity data, with details around:
 - assignment ID (unique ID per assignment - scraped from Insights Activity AssignmentId),
 - school ID (ID from Organization table),
 - course ID, 
 - section ID, 
 - instructor ID, 
 - student ID (from Person table),
 - assignment status ID (ID from dim_AssignmentStatus for student assignment status),
 - assigned date, 
 - assignment due date, and
 - student grade.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/fact_Assignment```.

In [58]:
dfInsights_aadgroup = oea.load('stage2/Refined/M365/v1.14/general/AadGroup')
dfInsights_aaduserpersonmap = oea.load('stage2/Refined/M365/v1.14/general/AadUserPersonMapping')
dfInsights_activity = oea.load('stage2/Refined/M365/v1.14/general/activity')
# lookup(s)
dfInsights_aadgroup_np = oea.load('stage2/Refined/M365/v1.14/sensitive/AadGroup_lookup')

StatementMeta(spark3p3sm, 59, 11, Finished, Available)

In [70]:
# NOTE: clean up below
# grab only the relevant columns from the Insights activity student assignment data
dfAssign = dfInsights_activity.filter(dfInsights_activity['AppName'] == 'Assignments').filter(dfInsights_activity['ActorRole'] == 'Student')
dfAssign = dfAssign.select('StartTime', 'ClassId', 'ActorId_pseudonym', 'AssignmentId', 'Action', 'DueDate', 'Grade').withColumnRenamed('StartTime', 'AssignmentStatusTime')
# join the Insights AADGroup_pseudo and _lookup table to the dfAssign table, to provide mapping of the hashed and non-hashed AADGroup Class IDs
dfInsights_aadgroup_np_ = dfInsights_aadgroup_np.select('ObjectId', 'ObjectId_pseudonym').withColumnRenamed('ObjectId', 'Id').withColumnRenamed('ObjectId_pseudonym', 'AADGroup_ClassId_pseudonym')
dfAssign = dfAssign.join(dfInsights_aadgroup_np_, dfAssign.ClassId == dfInsights_aadgroup_np_.Id, how='inner').drop('Id')
dfInsights_aadgroup_ = dfInsights_aadgroup.select('ObjectId_pseudonym', 'SectionId')
dfAssign = dfAssign.join(dfInsights_aadgroup_, dfAssign.AADGroup_ClassId_pseudonym == dfInsights_aadgroup_.ObjectId_pseudonym, how='inner') \
                            .drop('ClassId', 'AADGroup_ClassId_pseudonym', 'ObjectId_pseudonym')
# join the Insights AADUserPersonMapping table to get the student IDs
df_metadata = dfInsights_aaduserpersonmap.select('ObjectId_pseudonym', 'PersonId_pseudonym')
dfAssign = dfAssign.join(df_metadata, dfAssign.ActorId_pseudonym == df_metadata.ObjectId_pseudonym, how='inner').drop('ActorId_pseudonym', 'ObjectId_pseudonym')
dfAssign = dfAssign.withColumnRenamed('PersonId_pseudonym', 'StudentId_pseudonym')
# scrape the Insights activity data for when each Instructor assigned the assignment
df_metadata = dfInsights_activity.filter(dfInsights_activity['SignalType'] == 'AssignmentEvent').filter(dfInsights_activity['Action'] == 'Assigned')
df_metadata = df_metadata.select('StartTime', 'AssignmentId').withColumn('AssignedDate', F.to_date(F.col('StartTime')))
df_metadata = df_metadata.withColumnRenamed('AssignmentId', 'id').drop('StartTime')
dfAssign = dfAssign.join(df_metadata, dfAssign.AssignmentId == df_metadata.id, how='left').drop('id')
# assign a status ID to each student assignment status update
dfAssign = dfAssign.withColumn('AssignmentStatusId', F.when(F.col('Action') == 'Assigned', '1').otherwise(F.when(F.col('Action') == 'Visited', '2').otherwise(F.when(F.col('Action') == 'Submitted', '3').otherwise(F.when(F.col('Action') == 'Returned', '4')))))
dfAssign = dfAssign.drop('Action')
# join dfAssign with dfEnroll table to grab the school, course and instructor IDs
df_metadata = dfEnroll.select('SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym')
df_metadata = df_metadata.withColumnRenamed('SectionId', 'classId').withColumnRenamed('StudentId_pseudonym', 'id')
dfAssign = dfAssign.join(df_metadata, (dfAssign.StudentId_pseudonym == df_metadata.id) & (dfAssign.SectionId == df_metadata.classId), how='inner').drop('classId', 'id')
dfAssign = dfAssign.select('AssignmentId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'AssignmentStatusId', 'AssignmentStatusTime', 'AssignedDate', 'DueDate', 'Grade')
# finally, inject every instance of an assignment being assigned (Insights activity data doesn't originally assign each assignment to each student)
df_metadata = dfAssign.groupBy('AssignmentId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'AssignedDate', 'DueDate').count()
df_metadata = df_metadata.drop('count')
df_metadata = df_metadata.withColumn('AssignmentStatusTime', F.col('AssignedDate')).withColumn('AssignmentStatusTime', F.col('AssignmentStatusTime').cast(TimestampType()))
df_metadata = df_metadata.withColumn('AssignmentStatusId', F.lit('1')).withColumn('Grade', F.lit(None))
df_metadata = df_metadata.select('AssignmentId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'AssignmentStatusId', 'AssignmentStatusTime', 'AssignedDate', 'DueDate', 'Grade')
dfAssign = dfAssign.unionAll(df_metadata)
# drop the AssignedDate and DueDate columns, and separate the AssignmentStatusTime column into AssignmentStatusDate and AssignmentStatusTime
dfAssign = dfAssign.drop('AssignedDate', 'DueDate').withColumn('AssignmentStatusDate', F.to_date(F.col('AssignmentStatusTime'))).withColumn('AssignmentStatusTime', F.date_format('AssignmentStatusTime', 'HH:mm:ss'))
dfAssign = dfAssign.select('AssignmentId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'AssignmentStatusId', 'AssignmentStatusDate', 'AssignmentStatusTime', 'Grade')
display(dfAssign.limit(10))

StatementMeta(spark3p3sm, 59, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 22381c9b-6dd0-4dcd-bc8c-c085633cdb44)

In [71]:
# create a unique primary key, since publishing will drop duplicates, and we intentionally want Assignment Id's reoccuring
dfAssign = dfAssign.withColumn('AssignmentActivityId', F.concat(F.col('AssignmentStatusTime'),F.lit('_'),F.col('StudentId_pseudonym'),F.lit('_'),F.col('AssignmentId')))
dfAssign = dfAssign.select('AssignmentActivityId', 'AssignmentId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'AssignmentStatusId', 'AssignmentStatusDate', 'AssignmentStatusTime', 'Grade')

display(dfAssign.limit(10))

StatementMeta(spark3p3sm, 59, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, 436cf651-8042-4d9f-92db-6ab780ef9d02)

### Write to Stage 3

In [72]:
publish(dfAssign, 'stage2/Enriched/learning_analytics/v1.0/general/fact_Assignment', 'stage3/Published/learning_analytics/v1.0/general/fact_Assignment', primary_key='AssignmentActivityId')

StatementMeta(spark3p3sm, 59, 25, Finished, Available)

2023-01-11 19:35:06,814 - OEA - INFO - Number of new inbound rows processed: 30785


30785

## 4.) Build fact_Activity Table

Data aggregations curation on Insights activity data in the education system (as well as using a few other tables).

This table has one row per student per signal from the Insights activity data, with details around:
 - activity ID (unique ID per assignment - scraped from Insights Activity SignalId),
 - school ID (ID from Organization table),
 - course ID, 
 - section ID, 
 - instructor ID, 
 - student ID (from Person table),
 - signal type ID (ID from dim_SignalType for student signal type),
 - activity date (i.e. date of the Insights activity signal), 
 - activity start time (i.e. datetime of the Insights activity signal), and
 - source file extension (i.e. if an activity with file manipulation or submission, the suffix describing the type of file such as .pptx).

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/fact_Activity```.

In [11]:
dfInsights_aadgroup = oea.load('stage2/Refined/M365/v1.14/general/AadGroup')
dfInsights_aaduserpersonmap = oea.load('stage2/Refined/M365/v1.14/general/AadUserPersonMapping')
dfInsights_activity = oea.load('stage2/Refined/M365/v1.14/general/activity')
# lookup(s)
dfInsights_aadgroup_np = oea.load('stage2/Refined/M365/v1.14/sensitive/AadGroup_lookup')

StatementMeta(spark3p3sm, 113, 11, Finished, Available)

In [12]:
# join IDs with Person IDs rather than AADUser IDs from Insights activity data
dfActivity = dfInsights_activity.select('SignalType', 'StartTime', 'SignalId', 'ClassId', 'ActorId_pseudonym', 'ActorRole', 'SourceFileExtension')
df_metadata = dfInsights_aaduserpersonmap.select('ObjectId_pseudonym', 'PersonId_pseudonym')
dfActivity = dfActivity.join(df_metadata, dfActivity.ActorId_pseudonym == df_metadata.ObjectId_pseudonym, how='left').drop('ActorId_pseudonym', 'ObjectId_pseudonym').withColumnRenamed('PersonId_pseudonym', 'ActorId_pseudonym')
# isolate only student signals
dfActivity = dfActivity.filter(dfActivity['ActorRole'] == 'Student').drop('ActorRole')
dfActivity = dfActivity.withColumnRenamed('ActorId_pseudonym', 'StudentId_pseudonym')
# replace AADGroup IDs with Section IDs
dfInsights_aadgroup_np_ = dfInsights_aadgroup_np.select('ObjectId', 'ObjectId_pseudonym').withColumnRenamed('ObjectId', 'Id').withColumnRenamed('ObjectId_pseudonym', 'AADGroup_ClassId_pseudonym')
dfActivity = dfActivity.join(dfInsights_aadgroup_np_, dfActivity.ClassId == dfInsights_aadgroup_np_.Id, how='inner').drop('Id')
dfInsights_aadgroup_ = dfInsights_aadgroup.select('ObjectId_pseudonym', 'SectionId')
dfActivity = dfActivity.join(dfInsights_aadgroup_, dfActivity.AADGroup_ClassId_pseudonym == dfInsights_aadgroup_.ObjectId_pseudonym, how='inner') \
                            .drop('ClassId', 'AADGroup_ClassId_pseudonym', 'ObjectId_pseudonym')
# clean data as needed and join dfActivity with dfEnroll table to grab the school, course and instructor IDs
df_metadata = dfEnroll.select('SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym')
df_metadata = df_metadata.withColumnRenamed('SectionId', 'classId').withColumnRenamed('StudentId_pseudonym', 'id')
dfActivity = dfActivity.join(df_metadata, (dfActivity.StudentId_pseudonym == df_metadata.id)  & \
                            (dfActivity.SectionId == df_metadata.classId), how='inner').drop('classId', 'id')
dfActivity = dfActivity.withColumn('ActivityDate', F.to_date(F.col('StartTime')))
dfActivity = dfActivity.select('SignalId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'SignalType', 'ActivityDate', 'StartTime', 'SourceFileExtension')


StatementMeta(spark3p3sm, 113, 12, Finished, Available)

In [13]:
# NOTE: Clean up SignalType assignment/encoding process.
def SignalTypeId(SignalType):
    if SignalType == 'PostChannelMessage':
        res = '000001'
    elif SignalType == 'ReplyChannelMessage':
        res = '000002'
    elif SignalType == 'VisitTeamChannel':
        res = '000003'
    elif SignalType == 'ExpandChannelMessage':
        res = '000004'
    elif SignalType == 'ReactedWithEmoji':
        res = '000005'
    elif SignalType == 'Like':
        res = '000010'
    elif SignalType == 'Unlike':
        res = '000011'
    elif SignalType == 'FileAccessed':
        res = '000020'
    elif SignalType == 'FileModified':
        res = '000021'
    elif SignalType == 'FileDownloaded':
        res = '000022'
    elif SignalType == 'FileUploaded':
        res = '000023'
    elif SignalType == 'ShareNotificationRequested':
        res = '000030'
    elif SignalType == 'AddedToSharedWithMe':
        res = '000040'
    elif SignalType == 'CommentCreated':
        res = '000050'
    elif SignalType == 'CommentDeleted':
        res = '000051'
    elif SignalType == 'UserAtMentioned':
        res = '000060'
    elif SignalType == 'Reflect':
        res = '000100'
    elif SignalType == 'OneNotePageChanged':
        res = '001000'
    elif SignalType == 'SubmissionEvent':
        res = '020000'
    elif SignalType == 'AssignmentEvent':
        res = '010000'
    elif SignalType == 'CallRecordSummarized':
        res = '100000'
    else:
        res = ''
    return res
# define the function/dataType
new_f2 = F.udf(SignalTypeId, StringType())
  
# add the new column
dfActivity = dfActivity.withColumn('SignalTypeId', new_f2('SignalType'))
dfActivity = dfActivity.select('SignalId', 'SchoolId', 'CourseId', 'SectionId', 'InstructorId_pseudonym', 'StudentId_pseudonym', 'SignalTypeId', 'ActivityDate', 'StartTime', 'SourceFileExtension')
display(dfActivity.limit(10))

StatementMeta(spark3p3sm, 113, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, ab366e3b-21ae-44a9-bee9-0f784d569df4)

### Write to Stage 3

In [15]:
publish(dfActivity, 'stage2/Enriched/learning_analytics/v1.0/general/fact_Activity', 'stage3/Published/learning_analytics/v1.0/general/fact_Activity', primary_key='SignalId')

StatementMeta(spark3p3sm, 113, 15, Finished, Available)

2023-02-27 02:45:48,445 - OEA - INFO - Number of new inbound rows processed: 38318


38318